In [1]:
import pandas as pd
import numpy as np
import os
from IPython.display import display

data_dir = "../data/erin_data"

pd.set_option("display.max_columns", None)

# read in GA data
ga_data_path = os.path.join(data_dir, "ga_data.csv")
ga_data = pd.read_csv(ga_data_path)
ga_data.replace([" "], np.nan, inplace=True)

# weight variable is numeric but stored as object - convert to float
ga_data["dgaWeight"] = ga_data.dgaWeight.astype("float")

## Error with dgaGA cols:

for some reason "dgaGAType" , "dgaGATotal", and "dgaGASequence" have been converted from their respective integer values to take the values "Extracted": 1, "Filled": 2, "Sealed": 3

convert them back to correct values - dgaGAType maps to 1 = exodontia, 2 = comprehensive care - other two are integer fields

In [2]:
correct_tot_seq_vals = {
    "Extracted": 1,
    "Filled": 2,
    "Sealed": 3,
}

ga_data["dgaGATotal"] = ga_data["dgaGATotal"].map(correct_tot_seq_vals)
ga_data["dgaGATotal"] = ga_data["dgaGATotal"].astype("int")
ga_data["dgaGASequence"] = ga_data["dgaGASequence"].map(correct_tot_seq_vals)
ga_data["dgaGASequence"] = ga_data["dgaGASequence"].astype("int")

correct_type_vals = {
    "Extracted" : "exodontia",
    "Filled" : "comprehensive_care"
}

ga_data["dgaGAType"] = ga_data.dgaGAType.map(correct_type_vals)
ga_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ChildID            1088 non-null   int64  
 1   AgeMths            1088 non-null   int64  
 2   PregnancyID        1088 non-null   object 
 3   MotherID           1088 non-null   int64  
 4   FatherID           219 non-null    float64
 5   dgaGAType          1088 non-null   object 
 6   dgaGATotal         1088 non-null   int64  
 7   dgaGASequence      1088 non-null   int64  
 8   dgaWeight          915 non-null    float64
 9   dgaUR6             1087 non-null   object 
 10  dgaURE             1087 non-null   object 
 11  dgaURD             1085 non-null   object 
 12  dgaURC             1088 non-null   object 
 13  dgaURB             1088 non-null   object 
 14  dgaURA             1085 non-null   object 
 15  dgaULA             1086 non-null   object 
 16  dgaULB             1087 

In [3]:
# read in epi data
epi_data_path = os.path.join(data_dir, "epi_data.csv")
epi_data = pd.read_csv(epi_data_path)
epi_data.replace([" "], np.nan, inplace=True)
epi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ChildID            354 non-null    int64  
 1   PregnancyID        354 non-null    object 
 2   MotherID           354 non-null    int64  
 3   FatherID           111 non-null    float64
 4   AgeMths            354 non-null    int64  
 5   ddsdt              354 non-null    int64  
 6   ddsmt              354 non-null    int64  
 7   ddsft              354 non-null    int64  
 8   ddsdmft            354 non-null    int64  
 9   ddscareindex       354 non-null    int64  
 10  res6mcage          354 non-null    int64  
 11  res6mcbetterstart  354 non-null    object 
dtypes: float64(1), int64(9), object(2)
memory usage: 33.3+ KB


In [4]:
# read in merged data
merge_data_path = os.path.join(data_dir, "epi_ga_merge_data.csv")
merge_data = pd.read_csv(merge_data_path)
merge_data.replace([" "], np.nan, inplace=True)

# for some reason first two entries are blank
merge_data = merge_data.iloc[2:]

merge_data["ChildID"] = merge_data.ChildID.astype("int")

type_keys_to_vals = {
    1 : "exodontia",
    2 : "comprehensive_care"
}
merge_data["dgaGAType"] = merge_data.dgaGAType.astype("float")
merge_data["dgaGAType"] = merge_data.dgaGAType.map(type_keys_to_vals)

merge_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1416 entries, 2 to 1417
Data columns (total 77 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ChildID                            1416 non-null   int64  
 1   PregnancyID                        1399 non-null   object 
 2   MotherID                           1399 non-null   float64
 3   FatherID                           318 non-null    float64
 4   admincgender                       1399 non-null   object 
 5   agefm_fbqall                       311 non-null    float64
 6   agemm_mbqall                       1126 non-null   float64
 7   ben0mentst                         1122 non-null   object 
 8   bib6a02                            139 non-null    object 
 9   bib6b04                            139 non-null    object 
 10  bib6b05                            141 non-null    object 
 11  deminfeth3gpcomb                   1357 non-null   objec

## AgeMths

Age variable is problem - has been treated as one observation in merged dataset, but it isn't - age in relation to the Epi dataset is when kids were observed in Epi data - age in GA dataset is age when general anaesthetic was given - big problem for kids that overlap

doesn't really matter as all ages for kids in GA data have been fudged and appear as NA in merged data!

Also all Epi kids are either 65 or 66 months old which doesn't seem right and won't really tell us much

**Correct as follows:**

Epi data - bin the ages - they probably aren't any use anyway

only use age colum from GA data

merged and GA data match on the grouped key ["ChildID", "dgaGASequence"] (sequence is the order of anaesthetic for kids that have been put under multiple times) - merge the ages from the GA data as a new feature "dgaAgeMths - rename age variable in GA data to match

In [5]:
merge_data.drop("AgeMths", axis=1, inplace=True)
# merge AgeMths col from GA data into merged data
key_cols = ["ChildID", "dgaGASequence"]
merge_data = merge_data.merge(ga_data[key_cols + ["AgeMths"]],
                              how="left",
                              on=key_cols)

## duplicate entries

there are equvalent duplicated entries for individual children in GA data and merged data:


In [6]:
from IPython.display import display

three_ops = ga_data.dgaGATotal == 3
three_op_ids = ga_data[three_ops].ChildID.unique()

for id in three_op_ids[:2]:
    print("GA Entries:")
    display(ga_data[ga_data.ChildID == id]
            .sort_values(["dgaGASequence"])) #%%
    print("Merged Entries:")
    display(merge_data.loc[merge_data.ChildID == id, ga_data.columns]
            .sort_values(["dgaGASequence"])) #%%

GA Entries:


,ChildID,AgeMths,PregnancyID,MotherID,FatherID,dgaGAType,dgaGATotal,dgaGASequence,dgaWeight,dgaUR6,dgaURE,dgaURD,dgaURC,dgaURB,dgaURA,dgaULA,dgaULB,dgaULC,dgaULD,dgaULE,dgaUL6,dgaLL6,dgaLLE,dgaLLD,dgaLLC,dgaLLB,dgaLLA,dgaLRA,dgaLRB,dgaLRC,dgaLRD,dgaLRE,dgaLR6,dgaUR4,dgaUR1,dgaUL1,dgaUL4,dgaLL5,dgaLL4,dgaLL1,dgaLR1,dgaLR4,dgaLR5,res6mcage,res6mcbetterstart
0,12465,57,10328P1,310,NaN,comprehensive_care,3,1,NaN,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,Yes
2,12465,57,10328P1,310,NaN,comprehensive_care,3,2,NaN,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,Yes
1,12465,97,10328P1,310,NaN,comprehensive_care,3,3,39.0,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96,Yes


Merged Entries:


,ChildID,AgeMths,PregnancyID,MotherID,FatherID,dgaGAType,dgaGATotal,dgaGASequence,dgaWeight,dgaUR6,dgaURE,dgaURD,dgaURC,dgaURB,dgaURA,dgaULA,dgaULB,dgaULC,dgaULD,dgaULE,dgaUL6,dgaLL6,dgaLLE,dgaLLD,dgaLLC,dgaLLB,dgaLLA,dgaLRA,dgaLRB,dgaLRC,dgaLRD,dgaLRE,dgaLR6,dgaUR4,dgaUR1,dgaUL1,dgaUL4,dgaLL5,dgaLL4,dgaLL1,dgaLR1,dgaLR4,dgaLR5,res6mcage,res6mcbetterstart
1065,12465,57.0,10328P1,310.0,NaN,comprehensive_care,3.0,1.0,NaN,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1076,12465,57.0,NaN,NaN,NaN,comprehensive_care,3.0,2.0,NaN,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1082,12465,97.0,NaN,NaN,NaN,comprehensive_care,3.0,3.0,39.0,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


GA Entries:


,ChildID,AgeMths,PregnancyID,MotherID,FatherID,dgaGAType,dgaGATotal,dgaGASequence,dgaWeight,dgaUR6,dgaURE,dgaURD,dgaURC,dgaURB,dgaURA,dgaULA,dgaULB,dgaULC,dgaULD,dgaULE,dgaUL6,dgaLL6,dgaLLE,dgaLLD,dgaLLC,dgaLLB,dgaLLA,dgaLRA,dgaLRB,dgaLRC,dgaLRD,dgaLRE,dgaLR6,dgaUR4,dgaUR1,dgaUL1,dgaUL4,dgaLL5,dgaLL4,dgaLL1,dgaLR1,dgaLR4,dgaLR5,res6mcage,res6mcbetterstart
61,13149,66,11464P1,1431,NaN,comprehensive_care,3,1,NaN,Nil treatment,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Sealed,Sealed,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Sealed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,No
63,13149,66,11464P1,1431,NaN,comprehensive_care,3,2,NaN,Nil treatment,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Sealed,Sealed,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Sealed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,No
62,13149,97,11464P1,1431,NaN,comprehensive_care,3,3,NaN,Filled,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Filled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96,No


Merged Entries:


,ChildID,AgeMths,PregnancyID,MotherID,FatherID,dgaGAType,dgaGATotal,dgaGASequence,dgaWeight,dgaUR6,dgaURE,dgaURD,dgaURC,dgaURB,dgaURA,dgaULA,dgaULB,dgaULC,dgaULD,dgaULE,dgaUL6,dgaLL6,dgaLLE,dgaLLD,dgaLLC,dgaLLB,dgaLLA,dgaLRA,dgaLRB,dgaLRC,dgaLRD,dgaLRE,dgaLR6,dgaUR4,dgaUR1,dgaUL1,dgaUL4,dgaLL5,dgaLL4,dgaLL1,dgaLR1,dgaLR4,dgaLR5,res6mcage,res6mcbetterstart
1066,13149,66.0,11464P1,1431.0,NaN,comprehensive_care,3.0,1.0,NaN,Nil treatment,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Sealed,Sealed,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Sealed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1077,13149,66.0,NaN,NaN,NaN,comprehensive_care,3.0,2.0,NaN,Nil treatment,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Extracted,Sealed,Sealed,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Sealed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083,13149,97.0,NaN,NaN,NaN,comprehensive_care,3.0,3.0,NaN,Filled,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Extracted,Extracted,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Nil treatment,Filled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It seems something has gone wrong with the kids that have been given 3 GAs - the entries for the first and second GAs have been duplicated - all the obs for entries with sequence 1 or 2 are identical (other than the sequence obs itself)

probs something went wrong with data extract or method for counting kids that appear multiple times

remove for now:

In [7]:
duplicate_ga_entries = ga_data.dgaGASequence == 2
ga_data = ga_data[~duplicate_ga_entries]

duplicate_merged_entries = merge_data.dgaGASequence == 2
merge_data = merge_data[~duplicate_merged_entries]

## Test That Epi / GA data agree with merged data

EPI data is fine (we don't care about age cos we binned that)

In [8]:
epi_test_merge = epi_data.merge(merge_data, on="ChildID", how="left")

problem_idxs = []
for col in epi_test_merge.columns:
    if col[-2:] == "_x":
        x_col = col
        y_col = col[:-1] + "y"

        match_mask = epi_test_merge[x_col] == epi_test_merge[y_col]
        both_na_mask = epi_test_merge[x_col].isna() & epi_test_merge[y_col].isna()

        n_same = sum(match_mask)
        n_both_na =  sum(both_na_mask)
        n_same += n_both_na

        if n_same != len(epi_test_merge):
            print("=" * 80)
            print(f"problem with {col[:-2]}")
            print(f"{n_same - len(epi_test_merge)} columns dont match")
            print()
            problem_idxs += list(
                epi_test_merge[~match_mask & ~both_na_mask].index
            )

problem with AgeMths
-354 columns dont match



GA data not so much:

In [27]:
ga_test_merge = ga_data.merge(merge_data,
                              on=["ChildID", "AgeMths"],
                              how="left")

problem_idxs = {}
for col in ga_test_merge.columns:
    if col[-2:] == "_x":
        x_col = col
        y_col = col[:-1] + "y"

        match_mask = ga_test_merge[x_col] == ga_test_merge[y_col]
        both_na_mask = ga_test_merge[x_col].isna() & ga_test_merge[y_col].isna()

        n_same = sum(match_mask)
        n_both_na =  sum(both_na_mask)
        n_same += n_both_na

        if n_same != len(ga_test_merge):
            print("=" * 80)
            print(f"problem with {col[:-2]}")
            print(f"{n_same - len(ga_test_merge)} columns dont match")
            print()
            problem_idxs[col[:-2]] = list(
                ga_test_merge[~match_mask & ~both_na_mask].ChildID.values
            )

problem with PregnancyID
-6 columns dont match

problem with MotherID
-6 columns dont match

problem with FatherID
-1 columns dont match

problem with res6mcage
-24 columns dont match



Not too bothered about Mother / Pregnancy / Father ID at mo

* dgaWeight - has been rounded and there appear to have been rounding errors - replace with col from GA data
* res6mcage / res6mcbetterstart - like AgeMths have been overwritten in merged data and are all NaNs - just copy across from ga data where blank

In [25]:
ga_data.res6mcage

0        24
1        96
3        72
4        96
5       102
       ... 
1083     66
1084     60
1085     66
1086     48
1087     66
Name: res6mcage, Length: 1077, dtype: int64

In [24]:
np.round(ga_data.AgeMths / 6) * 6

0        60.0
1        96.0
3        72.0
4        96.0
5       114.0
        ...  
1083     66.0
1084     60.0
1085     72.0
1086     48.0
1087     72.0
Name: AgeMths, Length: 1077, dtype: float64

In [26]:
merge_data.drop("dgaWeight", axis=1, inplace=True)
# merge AgeMths col from GA data into merged data
ga_key_cols = ["ChildID", "AgeMths"]
merge_data = merge_data.merge(ga_data[ga_key_cols + ["dgaWeight"]],
                              how="left",
                              on=ga_key_cols)

mc_cols = ["res6mcage", "res6mcbetterstart"]
merge_data = merge_data.merge(ga_data[ga_key_cols + mc_cols],
                              how="left",
                              on=ga_key_cols,
                              suffixes=("", "_ga"))
mcage_is_na = merge_data.res6mcage.isna()
merge_data.loc[mcage_is_na, "res6mcage"] = merge_data[mcage_is_na].res6mcage_ga

bs_is_na = merge_data.res6mcbetterstart.isna()
merge_data.loc[bs_is_na, "res6mcbetterstart"] = (
    merge_data[bs_is_na].res6mcbetterstart_ga
)
merge_data.drop("res6mcbetterstart_ga", axis=1, inplace=True)

In [22]:
unk_cols = ['ddscareindex', 'GATx', 'Epi', 'has_bib1kc',
            'res6mcage', 'res6mcbetterstart']

rename_cols = {
    'admincgender': 'gender',
    'agefm_fbqall': 'father_age_at_q',
    'agemm_mbqall': 'mother_age_at_q',
    'ben0mentst': 'on_benefits',
    'bib6a02': 'describe_health_q',
    'bib6b04': 'has_diagnosis_q',
    'bib6b05': 'hospital_admission_q',
    'deminfeth3gpcomb': 'ethnicity',
    'edcont_eal': 'english_additional_lang',
    'edcont_lac': 'looked_after_child',
    'edcont_sen': 'special_ed_needs',
    'edu0fthede': 'father_highest_ed',
    'edu0mumede': 'mother_highest_ed',
    'eth0eth9gp': 'mother_ethnicity',
    'fbqageeduc': 'age_father_complete_ed',
    'fbqcountrybirth': 'father_birthplace',
    'imd_2010_decile_nat': 'imd_2010_decile',
    'mbqlcasep5gp': 'socio_economic_pos',
    'org0agemuk': 'age_mother_moved_uk',
    'org0mmubct': 'mother_birthplace',
    'qad0langua': 'questionaire_language',
    'Totalnoextractions': 'n_extractions',
    'totalnumberofprimaryextractions': 'n_primary_extract',
    'totalnumberofpermanentextractions': 'n_perm_extract',
    'AgeMths': 'age_at_ga',
    'ddsdt': 'decayed_teeth',
    'ddsmt':'missing_teeth',
    'ddsft': 'filled_teeth',
    'ddsdmft': 'dmft',
    'dgaGAType': 'type_of_ga',
    'dgaGATotal':'total_ga',
    'dgaGASequence': 'ga_sequence',
    'dgaWeight': 'weight_at_ga',
    'dgaUR6':'UR6',
    'dgaURE':'URE',
    'dgaURD':'URD',
    'dgaURC':'URC',
    'dgaURB':'URB',
    'dgaURA':'URA',
    'dgaULA':'ULA',
    'dgaULB':'ULB',
    'dgaULC':'ULC',
    'dgaULD':'ULD',
    'dgaULE':'ULE',
    'dgaUL6':'UL6',
    'dgaLL6':'LL6',
    'dgaLLE':'LLE',
    'dgaLLD':'LLD',
    'dgaLLC':'LLC',
    'dgaLLB':'LLB',
    'dgaLLA':'LLA',
    'dgaLRA':'LRA',
    'dgaLRB':'LRB',
    'dgaLRC':'LRC',
    'dgaLRD':'LRD',
    'dgaLRE':'LRE',
    'dgaLR6':'LR6',
    'dgaUR4':'UR4',
    'dgaUR1':'UR1',
    'dgaUL1':'UL1',
    'dgaUL4':'UL4',
    'dgaLL5':'LL5',
    'dgaLL4':'LL4',
    'dgaLL1':'LL1',
    'dgaLR1':'LR1',
    'dgaLR4':'LR4',
    'dgaLR5':'LR5'
}
# merge_data.rename(rename_cols, axis=1, inplace=True)
